<a href="https://colab.research.google.com/github/Bahaeddinferiani/Sentiment-analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q pyspark==3.1.2 spark-nlp

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Demarrage d'une session spark

In [ ]:
spark = sparknlp.start()

## 3. Nom du model


In [ ]:
MODEL_NAME='sentimentdl_use_imdb'


## 4. Importation du fichier json et transformation en un table 

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/reviews.json", "r")
reviews = f.read()
reviews = reviews.replace('"','')
reviews = reviews[1:]
reviews = reviews.strip()
reviews = reviews.split(",")
for i in reviews:
    i = i.strip()
text_list = reviews

## 5. Configuration du pipeline ( format, encodage du texte, selection du modele pré trainé)

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


## 6. Demarrage du pipeline

In [ ]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

## 7. Visualisation des résultats


In [ ]:

result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)
baha = result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment"))
baha.write.csv("/content/drive/MyDrive/Colab Notebooks/resultat")

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                                                                                                                                                                                 |sentiment|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|I am conflicted with this film and with Marvel.                                                                      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')